In [ ]:
import gc
import time
import torch


# pytorch가 기록하는 Peak 메모리 초기화
def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

# 메모리 최대 사용량 확인
def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    # 파이썬 garbage collection
    gc.collect()
    # torch 캐시 비우기
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    #기록 초기화
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

- 메모리 벤치마크를 위한 공통 함수 정의

In [2]:
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel



BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- 모델은 GPT2에서 가장큰 버전인 gpt2-xl을 사용함

In [ ]:
start_memory_tracking()

print(torch.cuda.is_available())

model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

In [ ]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

del model, test_input
cleanup()

- 모델 실행 후 GPU 메모리를 비우는 과정 ..

In [ ]:
# Then load pretrained weights

start_memory_tracking()

## BASE_CONFIG로 깡통 모델 생성
model = GPTModel(BASE_CONFIG)

## GPU로 옮김 (GPU 현재 메모리 6.4GB)
model.to(device)

# map_location=device 로 인하여 파일에서 읽은 가중치 데이터가 곧장 GPU로 가게됨
# (GPU 현재 메모리 12.8GB) (모델 초기 랜덤 Weight 6.4GB) + (불러온 Weights 6.4GB    )
model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

- 위에서 같은 CONFIG로 생성된 모델임에도 메모리를 두배를 쳐먹는걸 볼 수 있다.

In [ ]:
start_memory_tracking()

# 먼저 깡통 모델을 GPU에 넣음
model = GPTModel(BASE_CONFIG).to(device)

# 불러온 가중치를 CPU에 넣음
state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            # GPU에서 메모리를 점유하고 있는 깡통 모델의 가중치에 CPU에 있는 가중치를 복사함 
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

- 복사할 임시 가중치는 CPU에 저장한 후 GPU로 복사하는 과정을 통해 메모리를 아꼈다.

In [ ]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    # 함수 실행 전 메모리 재기
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            # 0.1 초마다 현재 메모리 사용량을 기록
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    # 메모리 감시 스레드 실행 (함수가 실행되는 와중에도 메모리 사용량을 기록하기 위해)
    t = Thread(target=monitor_memory)
    t.start()

    # 메모리 측정에 사용될 함수 실행
    func(*args, **kwargs)

    # Stop monitoring
    # 모니터링 종료
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb

- 쓰레드를 통해 작업이 시작 전, 끝난 후 두 순간의 메모리 사용량 뿐만 아니라 실행되는 도중 메모리 사용량을 체크할 수 있음

In [ ]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)

- 앞서 했던것처럼 깡통 모델 GPU 담기 => 불러온 가중치 CPU에 담기 => 
- 최대 메모리가 비슷한걸 볼 수 있다..

In [ ]:
def load_sequentially_with_meta():
    start_memory_tracking()


## meta 디바이스는 임시로 실제 할당되는 메모리 없이 데이터 불러오기 전까지 할당할 수 있는 device
    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

# meta에만 있었던 깡통 모델(메모리 점유 0)이 실제 GPU에 할당됨
    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

- meta device로 실제 가중치 로드 전까지 임시로 device객체를 부여 할 수 있다.
    - 테스트할떄나 임시로 Model 초기화 없이 구조만 보고 싶을때 좋을듯 ..

In [ ]:
def best_practices():
  # meta객체를 통해 깡통 모델 
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  # mmap=True 옵션을 통해 가상 메모리 주소랑 Mapping만 해둠..
  # load_state_dict 함수의 assign=True를 통해 불러온 가중치로 replace
  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

- 임시로 깡통모델을 meta divice에 할당
- 이후 torch.load 인자인 **mmap = True** 설정을 활용하여 가중치 파일을 메모리에 전부 로드하지 않고 가상 메모리 주소랑 Mapping만 해둠
- load_state_dict의 assign=True 옵션은 **파라미터를 Copy하는게 아닌 인자의 가중치로 Replace함**